### This file is for AFTER Google Colab updated on 2023-07-21
Important things to note:
1. Colab's Ubuntu upgraded from 20.04 LTS to 22.04 LTS
2. Colab's CUDA upgraded, and downgrading it was impossible due to Ubuntu's upgrade.

These changes required use of the latest MXNet package made for cuda 11.2, `mxnet-cu112`,
which worked on the newer version. Also, MXNet became officially archived.

In [3]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.3 LTS"
PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


Validating similar to https://mxnet.apache.org/versions/1.9.1/get_started/validate_mxnet.html

In [4]:
!pip install mxnet-cu112

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 MB 2.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [5]:
import mxnet as mx
print('number of gpus, expecting 1:',  mx.context.num_gpus())  # 1 gpu
a = mx.nd.ones((2, 3), mx.gpu())
b = a * 2 + 1
print(b.asnumpy())

number of gpus, expecting 1: 1
[[3. 3. 3.]
 [3. 3. 3.]]


In [6]:
print(b)  # should show it's on the gpu with @gpu(0)
print('numpy dtype:', b.asnumpy().dtype)  # should be float32


[[3. 3. 3.]
 [3. 3. 3.]]
<NDArray 2x3 @gpu(0)>
numpy dtype: float32


In [7]:
del a, b

# Importing other packages, and set context

In [8]:
import mxnet as mx
import numpy as np
from mxnet import npx
from mxnet.gluon import nn, rnn
npx.set_np()
from mxnet import gluon
from mxnet import autograd

import time
import math

In [9]:
# specify a context
ctx = mx.cpu()

In [10]:
# WE CAN DO GPU NOW!
ctx = mx.gpu()

In [11]:
import pandas as pd

import json

# Setting some data processing & NN properties variables used later

In [12]:
# Some variables for data stuff later on
SEQ_LENGTH = 7
NUM_NEIGHBORS = 3

TRAINING_CITIES_RANGE = 'TODO'
TESTING_CITIES_RANGE = 'TODO'

_DRIVE_ROOT_FLDR = 'drive/MyDrive/Colab Notebooks Project/'

STD_COLS = ['PM25', 'PM10', 'CO', 'NO2', 'O3_8', 'SO2']
y_feature = 'PM25'
x_features = list(set(STD_COLS) - {y_feature}) \
              + ['WS.max', 'WD.max']
print('sequence length:', SEQ_LENGTH)
print('number of neighbors:', NUM_NEIGHBORS)
print('y_feature:', y_feature)
print('x_features:', x_features)

TRAIN_CITIES_PERCENT = 0.25  # first 25% of the list (below) will be the training cities

# order randomized
citycodes_list = [
    131000,
    410100,
    370100,
    120000,
    130500,
    370800,
    130900,
    140400,
    130200,
    130600,
    410800,
    130400,
    140100,
    410600,
    140300,
    371700,
    410700,
    370300,
    371500,
    140500,
    410200,
    130100,
    131100,
    410500,
    371400,
    371600,
    410900,
    110000,
]
_cutoff = int( len(citycodes_list) * TRAIN_CITIES_PERCENT )
test_citycode_list = citycodes_list[:_cutoff]
train_citycode_list = citycodes_list[_cutoff:]
del _cutoff
print('number of total cities:', len(citycodes_list))
print(f'train cities ({len(train_citycode_list)}): ', train_citycode_list)
print(f'test cities ({len(test_citycode_list)}): ', test_citycode_list)

sequence length: 7
number of neighbors: 3
y_feature: PM25
x_features: ['PM10', 'CO', 'SO2', 'NO2', 'O3_8', 'WS.max', 'WD.max']
number of total cities: 28
train cities (21):  [140400, 130200, 130600, 410800, 130400, 140100, 410600, 140300, 371700, 410700, 370300, 371500, 140500, 410200, 130100, 131100, 410500, 371400, 371600, 410900, 110000]
test cities (7):  [131000, 410100, 370100, 120000, 130500, 370800, 130900]


# Google drive importing


In [13]:
!ls

sample_data


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!cp 'drive/MyDrive/Colab Notebooks Project/jingjinji.csv' .
!cp 'drive/MyDrive/Colab Notebooks Project/aq_.csv' .

# params2 contains the saved model parameters and potentially other model-dependent information
!cp -r 'drive/MyDrive/Colab Notebooks Project/params2' ./params2


In [17]:
# testing saving to file
with open('test_out.txt', 'w') as f:
    f.write('hello world')
!cp 'test_out.txt' 'drive/MyDrive/Colab Notebooks Project/test_out.txt'

# test read from that saved file
!cp 'drive/MyDrive/Colab Notebooks Project/test_out.txt' .
with open('test_out.txt', 'r') as f:
    line_read = f.read()
    print(line_read)
    assert line_read.rstrip() == 'hello world'

hello world


# Copy prepared_seqs from Google Drive

In [18]:
!cp -r 'drive/MyDrive/Colab Notebooks Project/prepared_seqs' .
# where traintype is like:
#    > 'together_train_y'
#    > 'together_train_allx'
#    > 'separate_train_y'
#    > 'separate_train_targetx'
#    > 'separate_train_neighborx'
#  and replace 'train' with 'test' for the other case
filename = 'prepared_seqs/{seqtype}_seqs_{num_neighbors}nbrs_seqlen{seq_length}.npy'

In [19]:
def _npload(seqtype):
    a = None
    with open(filename.format(seqtype=seqtype, num_neighbors=NUM_NEIGHBORS,
                              seq_length=SEQ_LENGTH), 'rb') as f:
        a = np.load(f)
        print(' dtype:', a.dtype)
    return a

import pickle
def _pload(name):
    a = None
    with open(f'prepared_seqs/{name}.pickle', 'rb') as f:
        a = pickle.load(f)
        print('',type(a))
    return a

together_cities_neighbors_dict = _pload('together_cities_neighbors_dict')
together_largestneighbordist = _pload('together_largestneighbordist')

together_train_y_seqs = _npload('together_train_y')
together_train_allx_seqs = _npload('together_train_allx')
together_test_y_seqs = _npload('together_test_y')
together_test_allx_seqs = _npload('together_test_allx')

separate_cities_neighbors_dict = _pload('separated_cities_neighbors_dict')
separate_largestneighbordist = _pload('separated_largestneighbordist')

separate_train_y_seqs = _npload('separated_train_y')
separate_train_targetx_seqs = _npload('separated_train_targetx')
separate_train_neighborx_seqs = _npload('separated_train_neighborx')
separate_test_y_seqs = _npload('separated_test_y')
separate_test_targetx_seqs = _npload('separated_test_targetx')
separate_test_neighborx_seqs = _npload('separated_test_neighborx')

 <class 'dict'>
 <class 'float'>
 dtype: float64
 dtype: float64
 dtype: float64
 dtype: float64
 <class 'dict'>
 <class 'float'>
 dtype: float64
 dtype: float64
 dtype: float64
 dtype: float64
 dtype: float64
 dtype: float64


In [21]:
print(together_train_y_seqs[1])
print(together_train_allx_seqs[1])

[[-0.7808177]]
[[ 2.12809977  0.69022504 -0.49137577  2.73496269  2.50106225  0.24137931
   2.81410282  3.05930314  1.93375321 -1.13657368  6.24819473  8.16901703
   0.21674877 -0.2270227   0.42444805  1.85701359  2.04495661  1.60935456
  -1.18888703  2.73496269  1.64228123  0.4137931   0.31883011  0.56748829
   1.34345352  1.16363914  1.17682302 -1.48532931  2.91986964  2.07167174
   0.33990148 -0.02907952  0.59649266]
 [-0.6987676  -0.98583466 -0.85756918 -0.40845544 -0.16115893  0.21674877
   0.64314432  0.38209345 -0.28297091 -0.99707143  0.8858932   0.95525641
   0.14778325 -0.24924492  0.42444805 -0.26725762 -0.46596676 -0.28297091
  -1.13657368  0.33117235  0.09647538  0.37438424  0.39660789  0.56748829
   0.12958425 -0.15002276  0.25769351 -1.01450921  1.07080015  1.04113451
   0.22660099 -0.02907952  0.59649266]
 [-0.24979455 -0.82363533 -0.77038027 -0.03864155  0.09647538  0.1773399
   0.29298973 -0.29968044 -0.76956889 -1.04938477  0.33117235 -0.07528083
   0.14778325 -0.260

# Plot test versus train cities on shapefile map

In [ ]:
!cp 'drive/MyDrive/Colab Notebooks Project/joined_cc_names_dict.pickle' .
!cp 'drive/MyDrive/Colab Notebooks Project/joined_cc_latlongs_dict.pickle' .
import pickle
with open('joined_cc_names_dict.pickle', 'rb') as f:
  cities_names_dict = pickle.load(f)
with open('joined_cc_latlongs_dict.pickle', 'rb') as f:
  cities_latlongs_dict = pickle.load(f)
print(cities_names_dict)
print(cities_latlongs_dict)

In [ ]:
!cp -r 'drive/MyDrive/Colab Notebooks Project/stanford-bw669kf8724-shapefile/' .

In [ ]:
# import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
# import numpy as np


N = 256
vals = np.ones((N, 4))
vals[:, 0] = np.linspace(1, 232/256, N)
vals[:, 1] = np.linspace(1, 252/256, N)
vals[:, 2] = np.linspace(1, 255/256, N)
newcmp = mcolors.ListedColormap(vals)
# [0,1] -> [white, very light blue]   used for coloring the jingjinji region(s)


# https://towardsdatascience.com/mapping-with-matplotlib-pandas-geopandas-and-basemap-in-python-d11b57ab5dac

fp = 'stanford-bw669kf8724-shapefile/bw669kf8724.shp'

map_df: gpd.GeoDataFrame = gpd.read_file(fp)


# set the Jing-Jin-Ji region provinces to be colored in blue, using the 'jingjinji_mask' column for the ListedColormap. 0 will give it white and 1 will give it the bluish color, set above.

map_df['jingjinji_mask'] = 0
jingjinji_mask = [i in (2.0, 10.0, 27.0) for i in map_df.id_1.values]  # Beijing Hebei Tianjin
map_df.loc[jingjinji_mask, 'jingjinji_mask'] = 1


mask = [(lambda i: i in [2.0, 10.0, 27.0  # Beijing Hebei Tianjin
                         , 23.0, 12.0, 1.0, 15.0, 31.0, 22.0, 25.0])(i) for i in map_df.id_1.values]  # Shandong Henan Anhui Jiangsu Zhejiang Shaanxi Shanxi
print(map_df[mask])
map_df = map_df[mask]


# plt.rcParams["figure.figsize"] = [16,9]
plt.rcParams["figure.figsize"] = [16 * 2/3, 9 * 2/3]  # slightly smaller but same aspect ratio, for text sizing

# preview
# dfplt = map_df.plot(edgecolor=u'gray', linewidth=0.5, ax=axes1, column='jingjinji_mask', cmap='Blues')
# dfplt = map_df.plot(edgecolor=u'gray', linewidth=0.5, ax=axes1, column='jingjinji_mask', cmap=newcmp)
dfplt = map_df.plot(edgecolor=u'gray', linewidth=0.5, column='jingjinji_mask', cmap=newcmp)


df3ll = {
    cc: cities_latlongs_dict[cc]
    for cc in set(train_citycode_list)
}
print(df3ll)
df4ll = {
    cc: cities_latlongs_dict[cc]
    for cc in set(test_citycode_list)
}
print(df4ll)


_swp_args = lambda a1, a2: (a2, a1)
dfplt.scatter(*_swp_args(*zip(*df3ll.values())),
              c='tab:green',
              s=15, label='train cities')
dfplt.scatter(*_swp_args(*zip(*df4ll.values())), marker='*',
              c='red',
              s=15, label='test cities')


dfplt.legend(loc = 'upper left')

# plt.show()
# no need for show, it shows by itself in the notebook.

# Simple (allx, y) CityDataSet

In [ ]:
class SimpleCityDataSet:

    # def __init__(self, city_seqs_dict: dict[tuple[np.ndarray, np.ndarray]], train_city_list, validate_city_list, batch_size = 8):
    #     self.batch_size = batch_size
    #     # concatenate the training city's sequences into X, y
    #     trainX, trainy = [], []
    #     for city in train_city_list:
    #         cX, cy = city_seqs_dict[city]
    #         trainX.append(cX)
    #         trainy.append(cy)
    #     self.trainX: np.ndarray = np.concatenate(trainX, axis=0)
    #     self.trainy: np.ndarray = np.concatenate(trainy, axis=0)
    #     # concatenate the validation city's sequences into X, y
    #     validX, validy = [], []
    #     for city in validate_city_list:
    #         cX, cy = city_seqs_dict[city]
    #         validX.append(cX)
    #         validy.append(cy)
    #     self.validX: np.ndarray = np.concatenate(validX, axis=0)
    #     self.validy: np.ndarray = np.concatenate(validy, axis=0)

    #     # make a dataloader out of these
    #     print("SimpleCityDataSet: concatenated train X.shape:", self.trainX.shape)  # feature sequences
    #     print("SimpleCityDataSet: concatenated train y.shape:", self.trainy.shape)  # labels
    #     print("SimpleCityDataSet: concatenated validate X.shape:", self.validX.shape)  # feature sequences
    #     print("SimpleCityDataSet: concatenated validate y.shape:", self.validy.shape)  # labels


    def __init__(self,
                 train_seqs_allX: np.ndarray, train_seqs_y: np.ndarray,
                 test_seqs_allX: np.ndarray, test_seqs_y: np.ndarray,
                 batch_size: int = 8):
        # assert train_seqs_allX.shape[0] == train_seqs_y.shape[0]
        # assert test_seqs_allX.shape[0] == test_seqs_y.shape[0]
        # assert train_seqs_allX.shape[1] == train_seqs_y.shape[1]
        # assert test_seqs_allX.shape[1] == test_seqs_y.shape[1]
        self.batch_size = batch_size
        self.trainX = train_seqs_allX  .astype(np.float32)
        self.trainy = train_seqs_y     .astype(np.float32)
        self.validX = test_seqs_allX   .astype(np.float32)
        self.validy = test_seqs_y      .astype(np.float32)
        # make a dataloader out of these
        print("SimpleCityDataSet: seqs train X.shape:", self.trainX.shape)  # feature sequences
        print("SimpleCityDataSet: seqs train y.shape:", self.trainy.shape)  # labels
        print("SimpleCityDataSet: seqs validate X.shape:", self.validX.shape)  # feature sequences
        print("SimpleCityDataSet: seqs validate y.shape:", self.validy.shape)  # labels


    # def concat_all_X(self):
    #     return np.concatenate((self.trainX, self.validX))
    # def concat_all_y(self):
    #     return np.concatenate((self.trainy, self.validy))


    def get_dataloader(self, train, last_batch = 'keep',
                       _batch_size_override = None, _dataloader_kwargs = {}) -> mx.gluon.data.DataLoader:

        # [feature seqs, labels]
        if train=='both':
          shuffle = False
          tensors = [self.concat_all_X(), self.concat_all_y()]
        else:
          shuffle = bool(train)
          tensors = [self.trainX, self.trainy] if train else [self.validX, self.validy]
        dataset = mx.gluon.data.ArrayDataset(*tensors)  # *[feature seqs, labels]
        # shuffle = (self.shuffle if _shuffle_override is None else bool(_shuffle_override))
        batch_size = (self.batch_size if _batch_size_override is None else int(_batch_size_override))
        return mx.gluon.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, last_batch=last_batch, **_dataloader_kwargs)


# Complex (targetx, nbgrx, y) CityDataSet

In [ ]:
class ComplexCityDataSet:

    def __init__(self,
                 train_seqs_targetX: np.ndarray, train_seqs_neighborX: np.ndarray, train_seqs_y: np.ndarray,
                 test_seqs_targetX: np.ndarray, test_seqs_neighborX: np.ndarray, test_seqs_y: np.ndarray,
                 batch_size: int = 8):
        self.batch_size = batch_size
        self.traintgtX = train_seqs_targetX    .astype(np.float32)
        self.trainngbrX = train_seqs_neighborX .astype(np.float32)
        self.trainy = train_seqs_y             .astype(np.float32)
        self.validtgtX = test_seqs_targetX     .astype(np.float32)
        self.validngbrX = test_seqs_neighborX  .astype(np.float32)
        self.validy = test_seqs_y              .astype(np.float32)
        # make a dataloader out of these
        print("ComplexCityDataSet: seqs train target X.shape:", self.traintgtX.shape)  # feature sequences
        print("ComplexCityDataSet: seqs train ngbr X.shape:", self.trainngbrX.shape)  # feature sequences
        print("ComplexCityDataSet: seqs train y.shape:", self.trainy.shape)  # labels
        print("ComplexCityDataSet: seqs validate target X.shape:", self.validtgtX.shape)  # feature sequences
        print("ComplexCityDataSet: seqs validate ngbr X.shape:", self.validngbrX.shape)  # feature sequences
        print("ComplexCityDataSet: seqs validate y.shape:", self.validy.shape)  # labels



    def get_dataloader(self, train, last_batch = 'keep',
                       _batch_size_override = None, _dataloader_kwargs = {}) -> mx.gluon.data.DataLoader:

        # [feature seqs, labels]
        if train=='both':
          shuffle = False
          tensors = [self.concat_all_X(), self.concat_all_y()]
        else:
          shuffle = bool(train)
          tensors = [self.traintgtX, self.trainngbrX, self.trainy] if train \
                        else [self.validtgtX, self.validngbrX, self.validy]
        dataset = mx.gluon.data.ArrayDataset(*tensors)  # *[feature seqs, labels]
        # shuffle = (self.shuffle if _shuffle_override is None else bool(_shuffle_override))
        batch_size = (self.batch_size if _batch_size_override is None else int(_batch_size_override))
        return mx.gluon.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, last_batch=last_batch, **_dataloader_kwargs)


# Utility functions (creating net, validation, graphing)

## create_net

create_net(hidden_size=..., num_layers=...) defined here.

Creates a Sequential net, of an LSTM and a Dense at the end

The smaller number of layers was inspired by [this SO question/answer](https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm)

In [ ]:
def create_net(hidden_size=32, num_layers=3, _out_dense_units=1):

  ## create net
  net = nn.Sequential()

  # use net's name_scope to give child Blocks appropriate names.
  with net.name_scope():

    # LSTM
    #  default layout is 'TNC', but 'NTC' avoids having to transpose the tensors before passing through
    #   'TNC' corresponds to input tensor shape (seq_length, batch_size, num_inputs)
    # note: LSTM has arguments state_clip_min and state_clip_max but they aren't working on cpu mxnet
    # lstm_layer = rnn.LSTM(hidden_size=10, num_layers=3, layout='NTC')
    lstm_layer = rnn.LSTM(hidden_size=hidden_size, num_layers=num_layers, layout='NTC')
    net.add(lstm_layer)

    # Dense:
    #  Inputs:  if flatten is True, data should be a tensor with shape (batch_size, x1, x2, ..., xn), where x1 * x2 * ... * xn is equal to in_units.
    #  Outputs:  if flatten is True, out will be a tensor with shape (batch_size, units).
    out_layer = nn.Dense(units=_out_dense_units, flatten=True)
    net.add(out_layer)

  return net


## save_net_to_file

In [ ]:
_DRIVE_ROOT_FLDR = 'drive/MyDrive/Colab Notebooks Project/'
def save_net_to_file(net, filename, filefolder='params2'):
    # generate bash command arguments
    file_path = filefolder + '/' + filename
    repr_rel_folder = '.' if (not filefolder or len(filefolder)==0) else repr('./' + filefolder)
    # save
    net.save_parameters(file_path)
    # copy out to drive and back
    !cp {repr(file_path)} {repr(_DRIVE_ROOT_FLDR + file_path)}
    !cp {repr(_DRIVE_ROOT_FLDR + file_path)} {repr_rel_folder}
    print(f'saved parameters to "{file_path}"')

## train with validation
```
train(model, trainer,
    train_dataloader, loss_fn,
    num_epochs, test_dataloader)
```



In [ ]:
## put testing into train function

def train(model: nn.Block, trainer: gluon.Trainer, train_dataloader,
          loss_fn, num_epochs,
          test_dataloader=None,
          model_is_net2=False,
          save_every: int = 0, save_filename_template = 'filename_e{epoch}_tloss{tloss:.7f}_vloss{vloss:.7f}.params',
          save_folder: str = None, save_epoch_offset = 0,
          starting_best_vloss = 0,
          _gradient_clip = True,
          _verbosedbg=True):
    '''returns list of training losses for each epoch.
    <br>save_filename_template gets passed args `epoch`, `tloss`, `vloss`.'''

    _do_saving = save_every > 0
    # _save_at = save_every - 1   # testing if _save_at == epoch % save_every

    _lowest_vloss_yet = starting_best_vloss
    _do_vloss_save = False
    # _vloss = None; _tloss = None

    train_losses = []
    test_losses = []
    train_start_time = time.time()

    for epoch in range(num_epochs):

        train_start_time = time.time()  # moved this from bottom to top of for-loop

        # keep a sum for averaging this epoch's loss
        epoch_loss_sum = 0.

        # Iterate over training data
        for idx, batchsandlabel in enumerate(train_dataloader):

            if not model_is_net2:
                batch, label = batchsandlabel

                # print(batch.shape)
                # print(batch)

                if len(batch.shape) < 3:
                    batch = batch.reshape(*batch.shape, 1)  # shape so that the inputsize is 1
                batch = batch.astype('float32')

                batch = batch.as_in_context(ctx)
                label = label.as_in_context(ctx)

                batch_size = batch.shape[0]

            else:
                targetbatch, ngbrbatch, label = batchsandlabel

                # print(targetbatch.shape)
                # print(ngbrbatch.shape)
                # print(batch)

                if len(targetbatch.shape) < 3:
                    targetbatch = targetbatch.reshape(*targetbatch.shape, 1)  # shape so that the inputsize is 1
                targetbatch = targetbatch.astype('float32')
                if len(ngbrbatch.shape) < 3:
                    ngbrbatch = ngbrbatch.reshape(*ngbrbatch.shape, 1)  # shape so that the inputsize is 1
                ngbrbatch = ngbrbatch.astype('float32')

                targetbatch = targetbatch.as_in_context(ctx)
                ngbrbatch = ngbrbatch.as_in_context(ctx)
                label = label.as_in_context(ctx)

                batch_size = targetbatch.shape[0]

                if idx==0:
                  # perform one assertion just to ensure batch_size is same
                  # from both input batched x's
                  assert targetbatch.shape[0] == ngbrbatch.shape[0]




            with autograd.record():  # train_mode defaults to True

                # print('debug: forward pass...')

                ## Forward pass
                if not model_is_net2:
                    predicted = model(batch)
                else:
                    predicted = model(targetbatch, ngbrbatch)


                ## Compute loss
                loss: mx.numpy.ndarray = loss_fn(predicted, label)

                if idx == 0:  # first run of loop
                  if _verbosedbg or (epoch == 0):  # print structure only once if verbose debug is False
                    # get a preview
                    # also set _params_prefix, used for debug messages later
                    print('weight params:', model.collect_params('.*weight'))
                    _params_prefix = model.collect_params('.*weight')._prefix


                ## my emergency debug stop if nan is found
                if math.isnan(loss[0]) \
                            or any(math.isnan(i) for i in loss) \
                            or math.isnan(model.collect_params('.*weight')[_params_prefix+'lstm0_l0_h2h_weight']._grad[0].min()):
                    print()
                    print(f'DEBUG force stopping at epoch {epoch} idx {idx}')
                    print(f'epoch_loss_sum: {epoch_loss_sum}')
                    print('loss:', loss)
                    print('train_losses array:', train_losses)
                    print('\nDEBUG more info:')
                    print(f'batch: {batch}')
                    print(f'label: {label}')
                    print(f'predicted: {predicted}')
                    print(f'some weights data and grads, BEFORE loss.backward()  (the _grad is not updated yet):')
                    _i2h_weight = model.collect_params('.*weight')[_params_prefix+'lstm0_l0_i2h_weight']
                    _i2h_weight_data = _i2h_weight._data.copy()
                    _i2h_weight_grad = _i2h_weight._grad.copy()
                    print('  ',_i2h_weight)
                    print(f'    i2h _data shape: {_i2h_weight_data[0].shape}  |  [0]  min() {_i2h_weight_data[0].min()} , max() {_i2h_weight_data[0].max()}')
                    print(f'    i2h _grad shape: {_i2h_weight_grad[0].shape}  |  [0]  min() {_i2h_weight_grad[0].min()} , max() {_i2h_weight_grad[0].max()}')
                    _h2h_weight = model.collect_params('.*weight')[_params_prefix+'lstm0_l0_h2h_weight']
                    _h2h_weight_data = _h2h_weight._data.copy()
                    _h2h_weight_grad = _h2h_weight._grad.copy()
                    print('  ',_h2h_weight)
                    print(f'    h2h _data shape: {_h2h_weight_data[0].shape}  |  [0]  min() {_h2h_weight_data[0].min()} , max() {_h2h_weight_data[0].max()}')
                    print(f'    h2h _grad shape: {_h2h_weight_grad[0].shape}  |  [0]  min() {_h2h_weight_grad[0].min()} , max() {_h2h_weight_grad[0].max()}')
                    print(f' again, epoch {epoch} idx {idx}')
                    # exit to stop (the program keeps saying nan afterwards if you let it run)
                    raise RuntimeError("nan encountered. See debug info printed above this error")


                # store loss
                epoch_loss_sum += float(loss.mean())

                # print('debug: loss stored. Backward loss.backward() ...')

                ## Backward pass  (gradients get updated)
                #  Note: The gradients didn't seem to update if this was outside autograd.record()
                #   which contradicts many examples
                loss.backward()


            # end autograd.record scope


            ## Optimize i.e. step the trainer

            if not _gradient_clip:
                trainer.step(batch_size)

            elif _gradient_clip:
                # attempt to perform gradient clipping
                if _verbosedbg: print(f' performing gradient clipping (epoch {epoch} idx {idx}, batch_size = batch.shape[0] = {batch_size})...')
                trainer.allreduce_grads()
                # https://github.com/apache/mxnet/issues/11508
                grads = [i.grad(ctx).as_nd_ndarray() for i in model.collect_params().values() if i._grad is not None]
                _total_norm = gluon.utils.clip_global_norm(arrays=grads, max_norm=1)
                if _verbosedbg: print(f'  debug: clip_global_norm\'s total norm was {_total_norm}')
                # https://nlp.gluon.ai/examples/language_model/train_language_model.html
                trainer.update(batch_size=batch_size)
                # trainer.update(batch_size=1)
                # trainer.step(1)

        # end for idx and batches (train)

        train_elapsed = time.time() - train_start_time

        train_losses.append(epoch_loss_sum/(idx+1))

        ## start testing, if applicable

        if test_dataloader:

            test_start_time = time.time()

            test_epoch_loss_sum = 0.

            # Iterate over test data
            # for t_idx, (batch, label) in enumerate(test_dataloader):
            for t_idx, batchsandlabel in enumerate(train_dataloader):

                if not model_is_net2:
                    batch, label = batchsandlabel

                    if len(batch.shape) < 3:
                      batch = batch.reshape(*batch.shape, 1)  # shape so that the inputsize is 1
                    batch = batch.astype('float32')

                    batch = batch.as_in_context(ctx)
                    label = label.as_in_context(ctx)

                    batch_size = batch.shape[0]

                else:
                    targetbatch, ngbrbatch, label = batchsandlabel

                    # print(targetbatch.shape)
                    # print(ngbrbatch.shape)
                    # print(batch)

                    if len(targetbatch.shape) < 3:
                        targetbatch = targetbatch.reshape(*targetbatch.shape, 1)  # shape so that the inputsize is 1
                    targetbatch = targetbatch.astype('float32')
                    if len(ngbrbatch.shape) < 3:
                        ngbrbatch = ngbrbatch.reshape(*ngbrbatch.shape, 1)  # shape so that the inputsize is 1
                    ngbrbatch = ngbrbatch.astype('float32')

                    targetbatch = targetbatch.as_in_context(ctx)
                    ngbrbatch = ngbrbatch.as_in_context(ctx)
                    label = label.as_in_context(ctx)

                    batch_size = targetbatch.shape[0]

                    if idx==0:
                      # perform one assertion just to ensure batch_size is same
                      # from both input batched x's
                      assert targetbatch.shape[0] == ngbrbatch.shape[0]

                ## Forward pass
                if not model_is_net2:
                    predicted = model(batch)
                else:
                    predicted = model(targetbatch, ngbrbatch)

                ## Compute loss
                loss: mx.numpy.ndarray = loss_fn(predicted, label)

                # store loss
                test_epoch_loss_sum += float(loss.mean())

            # end for idx and batches (test)


            test_elapsed = time.time() - test_start_time

            test_losses.append(test_epoch_loss_sum/(t_idx+1))

        # end if test_dataloader

        _test_epoch_loss_avg = 1000 if not test_dataloader else test_epoch_loss_sum/(t_idx+1)

        print('epoch [{}/{}], loss: {:.7f} , {:.4f} sec'.format(
                  epoch + 1, num_epochs, epoch_loss_sum/(idx+1), train_elapsed)
              + ('' if not test_dataloader else ' | validate loss: {:.7f} , {:.4f} sec'.format(
                  _test_epoch_loss_avg, test_elapsed))
        )

        if _test_epoch_loss_avg < _lowest_vloss_yet:
            _lowest_vloss_yet = _test_epoch_loss_avg
            _do_vloss_save = True

        if ( _do_vloss_save ) or ( _do_saving and epoch > 0 and (epoch % save_every == 0) ):
            save_net_to_file(model, save_filename_template.format( epoch=epoch+save_epoch_offset, tloss=train_losses[-1], vloss=('nan' if not test_dataloader else test_losses[-1]) ), \
                             **{'filefolder': p for p in [save_folder] if p is not None})  # extra optional kwarg
            _do_vloss_save = False

    #end for epoch loop

    print(f'train_losses: {train_losses}')
    if test_dataloader: print(f'test_losses: {test_losses}')

    return train_losses if not test_dataloader else (train_losses, test_losses)

## plt_losses

tlosses and vlosses would be lists of the losses

plt_losses(tlosses, vlosses=None, split_axis=True, title=None)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

def plt_losses(tlosses, vlosses=None, split_axis=True, title=None,
               x_offset=None):
    if not vlosses:
        split_axis=False
    axes2 = None
    plt.rcParams["figure.figsize"] = (6.5,4.5)
    axes: plt.Axes = plt.gca()
    fig = axes.figure
    axes.set_xlabel('epoch indexes')
    if split_axis:
        axes2 = axes.twinx()
        axes.set_ylabel('training losses')
        axes2.set_ylabel('validation losses')
    else:
        axes.set_ylabel('losses')

    # setup limits
    _tbottom=min(tlosses);  _ttop=max(tlosses)
    if vlosses: _vbottom=min(vlosses);  _vtop=max(vlosses)
    print('debug: testing:  tlosses min', min(tlosses));  print('debug: testing:  tlosses max', max(tlosses))
    if vlosses: print('debug: validate: vlosses min', min(vlosses));  print('debug: validate: vlosses max', max(vlosses))
    if split_axis:
        axes.set_ylim(bottom=_tbottom, top=_ttop)
        axes2.set_ylim(bottom=_vbottom, top=_vtop)
    else:
        if vlosses:
            axes.set_ylim(bottom=min(min(tlosses),min(vlosses)), top=max(max(tlosses),max(vlosses)))
        else:
            axes.set_ylim(bottom=min(tlosses), top=max(tlosses))

    # plot
    xs = range(len(tlosses)) if x_offset is None else x_offset+np.arange(len(tlosses))
    tlines = axes.plot(xs, tlosses, color='blue', marker='o', linestyle='dotted', label='training')
    if vlosses:
        if not split_axis:
            axes2 = axes  # graph on same axes
        xs = range(len(vlosses)) if x_offset is None else x_offset+np.arange(len(vlosses))
        vlines = axes2.plot(xs, vlosses, color='orange', marker='+', linestyle='dashed', label='validation')
    del xs

    # format ticks
    axes.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.4e'))
    if split_axis:
        axes2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.4e'))

    # legend
    if split_axis:
        lines = tlines + vlines
        plt.legend(lines, [l.get_label() for l in lines])
    else:
        plt.legend()
    if title: plt.title(title)
    plt.show()

# plt_losses(tlosses, vlosses, title='Split Y-axis Training and Validation')
# plt_losses(tlosses, vlosses, False, title='Common Y-axis Training and Validation')

# plt_losses(tlosses, title='Training only (one y-axis)')

## secs_to_hhmmss (hh:mm:ss.ms)

In [ ]:
def secs_to_hhmmss(seconds):
  return '{:02.0f}:{:02.0f}:{:02.3f}'.format(seconds//3600, seconds//60%60, seconds%60)

In [ ]:
# testing  secs_to_hhmmss
_timediff = 9317.4914
print( '{:.4f}, {:.4f}, {:.4f}'.format(_timediff/3600, _timediff/60, _timediff) )
print( '{:.4f}, {:.4f}, {:.4f}'.format(_timediff/3600, _timediff/60%60, _timediff%3600) )
print( '{:02.0f}:{:02.0f}:{:02.3f}'.format(_timediff//3600, _timediff//60%60, _timediff%60) )
print( '{:02.0f}:{:02.0f}:{:02.3f}'.format((_timediff+1e6)//3600, (_timediff+1e6)//60%60, (_timediff+1e6)%60) )
print( secs_to_hhmmss(_timediff) )
print( secs_to_hhmmss(_timediff+1e6) )

## create_trainer, create_loss_fn

create_trainer(net, type, lr, clip_grad)

create_loss_fn()

In [ ]:
def create_trainer(net, sgdtype='sgd', lr=0.1, clip_grad=5):
  '''if `clip_grad` param is passed `None`, then the clip_gradient arg will not be passed to gluon.Trainer'''
  if clip_grad is None:
    return gluon.Trainer(net.collect_params(), sgdtype, {'learning_rate': lr})
  return gluon.Trainer(net.collect_params(), sgdtype, {'learning_rate': lr, 'clip_gradient': clip_grad})

In [ ]:
def create_loss_fn():
  return gluon.loss.L2Loss()  # L2Loss is mean squared error (MSE)

Just calculate sqrt of MSE for analysis afterwards, same units.

# NN Creation

## Simple one (process all together in LSTM)

### create_net_1

create_net_1(hidden_size=..., num_layers=...) defined here.

Creates a Sequential net, of an LSTM and a Dense at the end

The smaller number of layers was inspired by [this SO question/answer](https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm)

In [ ]:
def create_net_1(hidden_size=32, num_layers=3, _out_dense_units=1):

  ## create net
  net = nn.Sequential()

  # use net's name_scope to give child Blocks appropriate names.
  with net.name_scope():

    # LSTM
    #  default layout is 'TNC', but 'NTC' avoids having to transpose the tensors before passing through
    #   'TNC' corresponds to input tensor shape (seq_length, batch_size, num_inputs)
    # note: LSTM has arguments state_clip_min and state_clip_max but they aren't working on cpu mxnet
    # lstm_layer = rnn.LSTM(hidden_size=10, num_layers=3, layout='NTC')
    lstm_layer = rnn.LSTM(hidden_size=hidden_size, num_layers=num_layers, layout='NTC')
    net.add(lstm_layer)

    # Dense:
    #  Inputs:  if flatten is True, data should be a tensor with shape (batch_size, x1, x2, ..., xn), where x1 * x2 * ... * xn is equal to in_units.
    #  Outputs:  if flatten is True, out will be a tensor with shape (batch_size, units).
    out_layer = nn.Dense(units=_out_dense_units, flatten=True)
    net.add(out_layer)

  return net

## Complex one (processing neighbors before LSTM)

### create_net_2

create_net_2(hidden_size=..., num_layers=..., ...) defined here.

Creates a Net to process Neighbors, then concat, then LSTM and a Dense at the end

The smaller number of layers was inspired by [this SO question/answer](https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm)

In [ ]:
## net
  #   target x
  # Dense
  # Dense
  #   Concat target x, neighbor x
  # LSTM
  # Dense
  #   out

from mxnet.gluon import Block
from mxnet import ndarray as F
# from mxnet import nd

class Net2NeighborPart(Block):

  def __init__(self, inner_dense_sizes, out_units, **kwargs):
    super(Net2NeighborPart, self).__init__(**kwargs)

    with self.name_scope():
      self.dense0 = nn.Dense(inner_dense_sizes[0],flatten=False) # for example, 32 units (intermediary)
      self.denseout = nn.Dense(out_units,flatten=False)    # for example, 9 units out

  def forward(self, x):
    x = self.dense0(x)
    # x = F.relu(self.dense0(x))
    return self.denseout(x)

class Net2(Block):

  def __init__(self, hidden_size, num_layers, out_dense_units,
               ngbr_inner_dense_sizes, ngbr_out_dense_units, **kwargs):
    super(Net2, self).__init__(**kwargs)

    with self.name_scope():
      self.neighbor_net = Net2NeighborPart(inner_dense_sizes = ngbr_inner_dense_sizes,
                                            out_units = ngbr_out_dense_units)

      # LSTM
      #  default layout is 'TNC', but 'NTC' avoids having to transpose the tensors before passing through
      #   'TNC' corresponds to input tensor shape (seq_length, batch_size, num_inputs)
      # note: LSTM has arguments state_clip_min and state_clip_max but they aren't working on cpu mxnet
      # lstm_layer = rnn.LSTM(hidden_size=10, num_layers=3, layout='NTC')
      self.lstm0 = rnn.LSTM(hidden_size=hidden_size, num_layers=num_layers, layout='NTC')

      self.densefinal = nn.Dense(out_dense_units)


  def forward(self, targetX, ngbrX):
    ngbrX = self.neighbor_net(ngbrX)

    # concat to make the LSTM input all together now

    x = F.np.concatenate((targetX, ngbrX),axis=2)

    x = self.lstm0(x)

    return self.densefinal(x)


def create_net_2(hidden_size=32, num_layers=3, _out_dense_units=1,
                 _ngbr_inner_dense_sizes=[32], _ngbr_out_dense_units=9):

  net = Net2(hidden_size = hidden_size, num_layers = num_layers,
             out_dense_units = _out_dense_units,
             ngbr_inner_dense_sizes = _ngbr_inner_dense_sizes,
             ngbr_out_dense_units = _ngbr_out_dense_units)

  #   # LSTM
  #   #  default layout is 'TNC', but 'NTC' avoids having to transpose the tensors before passing through
  #   #   'TNC' corresponds to input tensor shape (seq_length, batch_size, num_inputs)
  #   # note: LSTM has arguments state_clip_min and state_clip_max but they aren't working on cpu mxnet
  #   # lstm_layer = rnn.LSTM(hidden_size=10, num_layers=3, layout='NTC')
  #   lstm_layer = rnn.LSTM(hidden_size=hidden_size, num_layers=num_layers, layout='NTC')
  #   net.add(lstm_layer)

  #   # Dense:
  #   #  Inputs:  if flatten is True, data should be a tensor with shape (batch_size, x1, x2, ..., xn), where x1 * x2 * ... * xn is equal to in_units.
  #   #  Outputs:  if flatten is True, out will be a tensor with shape (batch_size, units).
  #   out_layer = nn.Dense(units=_out_dense_units, flatten=True)
  #   net.add(out_layer)

  return net


# Random Forest Tree (to be baseline comparison)

In [ ]:
# SEE OTHER FILE FOR Random Forest Tree

# Running tests for Simple

In [ ]:
print(together_train_y_seqs.shape)
print(together_train_allx_seqs.shape)
print(together_test_y_seqs.shape)
print(together_test_allx_seqs.shape)

In [ ]:
data = SimpleCityDataSet(together_train_allx_seqs, together_train_y_seqs,
                            together_test_allx_seqs, together_test_y_seqs)
print(data)
dataloader = data.get_dataloader(train=True)
_firstitem = next(iter(dataloader))
print('first item of dataloader:')
print(_firstitem)
print('expecting 2 items:', len(_firstitem))
print('shapes:')
print(_firstitem[0].shape, _firstitem[1].shape)
print('should be shaped like (batch_size, seq_length, num_imputs)')

In [ ]:
data = SimpleCityDataSet(together_train_allx_seqs, together_train_y_seqs,
                            together_test_allx_seqs, together_test_y_seqs)


In [ ]:
print(together_train_y_seqs.shape)
print(together_train_y_seqs[:,0,:].shape)
print(together_train_y_seqs[0].shape)
print(together_train_y_seqs[0])
print(together_train_y_seqs[:,0,:])
print('Note: the start of the sequences [:, 0, :] are not guaranteed to be truly consecutive'\
      +'\n because some sequences during preparation were cut out due to the data being non-consecutive')

## hidden size 32 (see function definition for reason!), num_layers 3

In [ ]:
net = create_net_1(hidden_size=32, num_layers=3)

net.initialize(mx.init.Xavier(), ctx=ctx)

print('net:');  print(net)

In [ ]:
print(data)
train_dataloader: gluon.data.DataLoader = data.get_dataloader(train=True)
val_dataloader: gluon.data.DataLoader = data.get_dataloader(train=False)

In [ ]:
## trainer for net
trainer = create_trainer(net)
loss_fn = create_loss_fn()


In [ ]:
## train, test
_stime = time.time()
tlosses, vlosses = train(net, trainer, train_dataloader, loss_fn, num_epochs=40, test_dataloader=val_dataloader, _verbosedbg=False)
_timediff = time.time() - _stime
print('total time diff for train() call, all epochs: {:.4f} sec (hh:mm:ss {})'.format(_timediff, secs_to_hhmmss(_timediff)))

# save net params in case Google Colab session ends
file_name = "params2/test_net_1.params"
net.save_parameters(file_name)
# copy out to drive and back
!cp 'params2/test_net_1.params' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_1.params'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_1.params' ./params2
print(f'saved parameters to "{file_name}"')

# plot losses
plt_losses(tlosses, vlosses)
plt_losses(tlosses, vlosses, split_axis=False)


In [ ]:
net = create_net_1(hidden_size=32, num_layers=3)
net.initialize(mx.init.Xavier(), ctx=ctx)
print('net:');  print(net)
## trainer for net
trainer = create_trainer(net)
loss_fn = create_loss_fn()

## train, test
_stime = time.time()
tlosses, vlosses = train(net, trainer, train_dataloader, loss_fn, num_epochs=10, test_dataloader=val_dataloader, _verbosedbg=False)
_timediff = time.time() - _stime
print('total time diff for train() call, all epochs: {:.4f} sec (hh:mm:ss {})'.format(_timediff, secs_to_hhmmss(_timediff)))
# plot losses
plt_losses(tlosses, vlosses)
plt_losses(tlosses, vlosses, split_axis=False)


### again

In [ ]:
net = create_net_1(hidden_size=32, num_layers=3)
net.initialize(mx.init.Xavier(), ctx=ctx)
print('net:');  print(net)
## trainer for net
trainer = create_trainer(net)
loss_fn = create_loss_fn()

In [ ]:
## train, test
_stime = time.time()
tlosses, vlosses = train(net, trainer, train_dataloader, loss_fn, num_epochs=50, test_dataloader=val_dataloader, _verbosedbg=False)
_timediff = time.time() - _stime
print('total time diff for train() call, all epochs: {:.4f} sec (hh:mm:ss {})'.format(_timediff, secs_to_hhmmss(_timediff)))

# save net params in case Google Colab session ends
file_name = "params2/test_net_1_e{epoch}_tl{tloss:.4f}_vl{vloss:.4f}.params".format(
    epoch=len(tlosses), tloss=tlosses[-1], vloss=vlosses[-1]
)
net.save_parameters(file_name)
# copy out to drive and back
!cp {repr(file_name)} {repr('drive/MyDrive/Colab Notebooks Project/'+file_name)}
!cp {repr('drive/MyDrive/Colab Notebooks Project/'+file_name)} ./params2
print(f'saved parameters to "{file_name}"')
# and as ... _latest .params
!cp {repr(file_name)} 'params2/test_net_1_latest.params'
!cp 'params2/test_net_1_latest.params' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_1_latest.params'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_1_latest.params' ./params2
print(f'copied parameters to "params2/test_net_1_latest.params"')

# plot losses
plt_losses(tlosses, vlosses)
plt_losses(tlosses, vlosses, split_axis=False)

In [ ]:
# save old losses
with open('params2/test_net_1_losses.json', 'w') as f:
  json.dump({'train_losses':tlosses, 'test_losses':vlosses}, f)
!cp 'params2/test_net_1_losses.json' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_1_losses.json'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_1_losses.json' ./params2

### again! load from saved

In [ ]:
!cp -r 'drive/MyDrive/Colab Notebooks Project/params2/' .
# open previous losses
with open('params2/test_net_1_losses.json', 'r') as f:
  _lossdict = json.load(f)
prev_tlosses = _lossdict['train_losses'] #['tlosses']
prev_vlosses = _lossdict['test_losses'] #['vlosses']
del _lossdict

In [ ]:
print(prev_tlosses); print(prev_vlosses)
print(len(prev_tlosses), 'previous training losses')
print(len(prev_vlosses), 'previous testing losses')
plt_losses(prev_tlosses, prev_vlosses, split_axis=False)
if len(prev_tlosses) >= 100:
    plt_losses(prev_tlosses[-100:], prev_vlosses[-100:], split_axis=False,
               x_offset=len(prev_tlosses)-100,
               title='Last 100 losses')

In [ ]:
net = create_net_1(hidden_size=32, num_layers=3)
print('net:');  print(net)
# reload from saved params
net.load_parameters('params2/test_net_1_latest.params', ctx=ctx)
print('net:');  print(net)

## trainer for net
trainer = create_trainer(net)
loss_fn = create_loss_fn()

In [ ]:
train_dataloader: gluon.data.DataLoader = data.get_dataloader(train=True)
val_dataloader: gluon.data.DataLoader = data.get_dataloader(train=False)

In [ ]:
_epoch_offset = len(prev_tlosses)+1
print('epoch offset used: ', _epoch_offset)
## train, test
_stime = time.time()
# tlosses, vlosses = train(net, trainer, train_dataloader, loss_fn, num_epochs=25, test_dataloader=val_dataloader, _verbosedbg=False)
tlosses, vlosses = train(net, trainer, train_dataloader, loss_fn, num_epochs=50, test_dataloader=val_dataloader,
                         save_every=10,
                         save_epoch_offset=_epoch_offset,
                         save_filename_template='test_net_2_e{epoch}_tl{tloss:.4f}_vl{vloss:.4f}.params',
                         save_folder = 'params2',
                         starting_best_vloss = min(prev_vlosses),  # <----  also saving when new best validation loss achieved
                         _verbosedbg=False)
_timediff = time.time() - _stime
print('total time diff for train() call, all epochs: {:.4f} sec (hh:mm:ss {})'.format(_timediff, secs_to_hhmmss(_timediff)))

# save net params in case Google Colab session ends
file_name = "params2/test_net_1_e{epoch}_tl{tloss:.4f}_vl{vloss:.4f}.params".format(
    epoch=len(prev_tlosses)+len(tlosses), tloss=tlosses[-1], vloss=vlosses[-1]
)
net.save_parameters(file_name)
# copy out to drive and back
!cp {repr(file_name)} {repr('drive/MyDrive/Colab Notebooks Project/'+file_name)}
!cp {repr('drive/MyDrive/Colab Notebooks Project/'+file_name)} ./params2
print(f'saved parameters to "{file_name}"')
# and as ... _latest .params
!cp {repr(file_name)} 'params2/test_net_1_latest.params'
!cp 'params2/test_net_1_latest.params' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_1_latest.params'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_1_latest.params' ./params2
print(f'copied parameters to "params2/test_net_1_latest.params"')

# plot losses
plt_losses(prev_tlosses + tlosses, prev_vlosses + vlosses)
plt_losses(prev_tlosses + tlosses, prev_vlosses + vlosses, split_axis=False)

In [ ]:
# plt_losses(tlosses[:50]+tlosses[100:], vlosses[:50]+vlosses[100:], split_axis=False)
# plt_losses(tlosses, vlosses, split_axis=False)

In [ ]:
# tlosses = tlosses[:50] + tlosses[100:]
# vlosses = vlosses[:50] + vlosses[100:]

In [ ]:
print(len(tlosses))
print(len(prev_tlosses))

In [ ]:
if len(prev_tlosses) >= 100:
    plt_losses(prev_tlosses[-100:], prev_vlosses[-100:], split_axis=False,
               x_offset=len(prev_tlosses)-100,
               title='Last 100 losses')

In [ ]:
# combine old with new losses
tlosses = prev_tlosses + tlosses
vlosses = prev_vlosses + vlosses
# print how many now
print(len(tlosses), 'now total training losses')
print(len(vlosses), 'now total testing losses')
# save losses
with open('params2/test_net_1_losses.json', 'w') as f:
  json.dump({'train_losses':tlosses, 'test_losses':vlosses}, f)
!cp 'params2/test_net_1_losses.json' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_1_losses.json'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_1_losses.json' ./params2

# Running tests for Complex

In [ ]:
print(separate_train_y_seqs.shape)
print(separate_train_targetx_seqs.shape)
print(separate_train_neighborx_seqs.shape)
print(separate_test_y_seqs.shape)
print(separate_test_targetx_seqs.shape)
print(separate_test_neighborx_seqs.shape)

In [ ]:
data2 = ComplexCityDataSet(separate_train_targetx_seqs, separate_train_neighborx_seqs, separate_train_y_seqs,
                            separate_test_targetx_seqs, separate_test_neighborx_seqs, separate_test_y_seqs)
print(data2)
dataloader2 = data2.get_dataloader(train=True)
_firstitem = next(iter(dataloader2))
print('first item of dataloader2:')
print(_firstitem)
print('expecting 3 items:', len(_firstitem))
print('shapes:')
print(_firstitem[0].shape, _firstitem[1].shape, _firstitem[2].shape)
print('should be shaped like (batch_size, seq_length, num_imputs) ; third one is the y')

## net2

In [ ]:
net2 = create_net_2(hidden_size=32, num_layers=3,
                    _ngbr_inner_dense_sizes=[32], _ngbr_out_dense_units=9)
net2.initialize(mx.init.Xavier(), ctx=ctx)
print('net2:');  print(net2)
## trainer for net
trainer = create_trainer(net2)
loss_fn = create_loss_fn()

In [ ]:
train_dataloader2: gluon.data.DataLoader = data2.get_dataloader(train=True)
val_dataloader2: gluon.data.DataLoader = data2.get_dataloader(train=False)

In [ ]:
## train, test
_stime = time.time()
tlosses, vlosses = train(net2, trainer, train_dataloader2, loss_fn, num_epochs=50, test_dataloader=val_dataloader2,
                         model_is_net2=True,  # <----
                         _verbosedbg=False)
_timediff = time.time() - _stime
print('total time diff for train() call, all epochs: {:.4f} sec (hh:mm:ss {})'.format(_timediff, secs_to_hhmmss(_timediff)))

# save net params in case Google Colab session ends
file_name = "params2/test_net_2.params"
net2.save_parameters(file_name)
# copy out to drive and back
!cp 'params2/test_net_2.params' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2.params'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2.params' ./params2
print(f'saved parameters to "{file_name}"')

# plot losses
plt_losses(tlosses, vlosses)
plt_losses(tlosses, vlosses, split_axis=False)

In [ ]:
# save old losses
with open('params2/test_net_2_losses.json', 'w') as f:
  json.dump({'train_losses':tlosses, 'test_losses':vlosses})
!cp 'params2/test_net_2_losses.json' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2_losses.json'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2_losses.json' ./params2

### reload it from its saved file

In [ ]:
!cp -r 'drive/MyDrive/Colab Notebooks Project/params2/' .
# open previous losses
with open('params2/test_net_2_losses.json', 'r') as f:
  _lossdict = json.load(f)
prev_tlosses = _lossdict['train_losses'] #['tlosses']
prev_vlosses = _lossdict['test_losses'] #['vlosses']
del _lossdict

In [ ]:
print(prev_tlosses); print(prev_vlosses)
print(len(prev_tlosses), 'previous training losses')
print(len(prev_vlosses), 'previous testing losses')

In [ ]:
net2 = create_net_2(hidden_size=32, num_layers=3,
                    _ngbr_inner_dense_sizes=[32], _ngbr_out_dense_units=9)
# net2.initialize(mx.init.Xavier(), ctx=ctx)
print('net2:');  print(net2)
# reload from saved params
net2.load_parameters('params2/test_net_2.params', ctx=ctx)
print('net2:');  print(net2)

## trainer for net
trainer = create_trainer(net2)
loss_fn = create_loss_fn()

In [ ]:
train_dataloader2: gluon.data.DataLoader = data2.get_dataloader(train=True)
val_dataloader2: gluon.data.DataLoader = data2.get_dataloader(train=False)

In [ ]:
## train, test
_stime = time.time()
tlosses, vlosses = train(net2, trainer, train_dataloader2, loss_fn, num_epochs=50, test_dataloader=val_dataloader2,
                         model_is_net2=True,  # <----
                         _verbosedbg=False)
_timediff = time.time() - _stime
print('total time diff for train() call, all epochs: {:.4f} sec (hh:mm:ss {})'.format(_timediff, secs_to_hhmmss(_timediff)))

# save net params in case Google Colab session ends
file_name = "params2/test_net_2.params"
net2.save_parameters(file_name)
# copy out to drive and back
!cp 'params2/test_net_2.params' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2.params'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2.params' ./params2
print(f'saved parameters to "{file_name}"')

# plot losses
plt_losses(prev_tlosses + tlosses, prev_vlosses + vlosses)
plt_losses(prev_tlosses + tlosses, prev_vlosses + vlosses, split_axis=False)

In [ ]:
# combine old with new losses
tlosses = prev_tlosses + tlosses
vlosses = prev_vlosses + vlosses

print(len(tlosses), 'now total training losses')
print(len(vlosses), 'now total testing losses')

# save losses
with open('params2/test_net_2_losses.json', 'w') as f:
  json.dump({'train_losses':tlosses, 'test_losses':vlosses}, f)
!cp 'params2/test_net_2_losses.json' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2_losses.json'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2_losses.json' ./params2

### again! more!

In [ ]:
# again!
!cp -r 'drive/MyDrive/Colab Notebooks Project/params2/' .
# open previous losses
with open('params2/test_net_2_losses.json', 'r') as f:
  _lossdict = json.load(f)
prev_tlosses = _lossdict['train_losses']
prev_vlosses = _lossdict['test_losses']
del _lossdict

print(prev_tlosses); print(prev_vlosses)
print(len(prev_tlosses), 'previous training losses')
print(len(prev_vlosses), 'previous testing losses')

print('OLD (PREVIOUS) LOSSES:')
plt_losses(prev_tlosses, prev_vlosses, split_axis=False)

In [ ]:
net2 = create_net_2(hidden_size=32, num_layers=3,
                    _ngbr_inner_dense_sizes=[32], _ngbr_out_dense_units=9)
print('net2:');  print(net2)
# reload from saved params
net2.load_parameters('params2/test_net_2_latest.params', ctx=ctx)
print('net2:');  print(net2)

## trainer for net
trainer = create_trainer(net2)
loss_fn = create_loss_fn()

In [ ]:
train_dataloader2: gluon.data.DataLoader = data2.get_dataloader(train=True)
val_dataloader2: gluon.data.DataLoader = data2.get_dataloader(train=False)

In [ ]:
_epoch_offset = len(prev_tlosses)+1
print('epoch offset used: ', _epoch_offset)
## train, test
_stime = time.time()
tlosses, vlosses = train(net2, trainer, train_dataloader2, loss_fn, num_epochs=100, test_dataloader=val_dataloader2,
                         model_is_net2=True,  # <----
                         save_every=10,
                         save_epoch_offset=_epoch_offset,
                         save_filename_template='test_net_2_e{epoch}_tl{tloss:.4f}_vl{vloss:.4f}.params',
                         save_folder = 'params2',
                         starting_best_vloss = min(prev_vlosses),  # <----  also saving when new best validation loss achieved
                         _verbosedbg=False)
_timediff = time.time() - _stime
print('total time diff for train() call, all epochs: {:.4f} sec (hh:mm:ss {})'.format(_timediff, secs_to_hhmmss(_timediff)))

# save net params in case Google Colab session ends
file_name = "params2/test_net_2_e{epoch}_tl{tloss:.4f}_vl{vloss:.4f}.params".format(
    epoch=len(prev_tlosses)+len(tlosses), tloss=tlosses[-1], vloss=vlosses[-1]
)
net2.save_parameters(file_name)
# copy out to drive and back
!cp {repr(file_name)} {repr('drive/MyDrive/Colab Notebooks Project/'+file_name)}
!cp {repr('drive/MyDrive/Colab Notebooks Project/'+file_name)} ./params2
print(f'saved parameters to "{file_name}"')
# and as ... _latest .params
!cp {repr(file_name)} 'params2/test_net_2_latest.params'
!cp 'params2/test_net_2_latest.params' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2_latest.params'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2_latest.params' ./params2
print(f'copied parameters to "params2/test_net_2_latest.params"')

# plot losses
plt_losses(prev_tlosses + tlosses, prev_vlosses + vlosses)
plt_losses(prev_tlosses + tlosses, prev_vlosses + vlosses, split_axis=False)

In [ ]:
# combine old with new losses
tlosses = prev_tlosses + tlosses
vlosses = prev_vlosses + vlosses
# print how many
print(len(tlosses), 'now total training losses')
print(len(vlosses), 'now total testing losses')
# SAVE all losses
with open('params2/test_net_2_losses.json', 'w') as f:
  json.dump({'train_losses':tlosses, 'test_losses':vlosses}, f)
!cp 'params2/test_net_2_losses.json' 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2_losses.json'
!cp 'drive/MyDrive/Colab Notebooks Project/params2/test_net_2_losses.json' ./params2

In [ ]:
if len(tlosses) >= 100:
    plt_losses(tlosses[-100:], vlosses[-100:], split_axis=False,
               x_offset=len(tlosses)-100,
               title='Last 100 t&v losses')

# Graph predictions over real

In [ ]:
# again!
!cp -r 'drive/MyDrive/Colab Notebooks Project/params2/' .
# open previous losses
with open('params2/test_net_2_losses.json', 'r') as f:
  _lossdict = json.load(f)
prev_tlosses = _lossdict['train_losses']
prev_vlosses = _lossdict['test_losses']
del _lossdict

print(prev_tlosses); print(prev_vlosses)
print(len(prev_tlosses), 'previous training losses')
print(len(prev_vlosses), 'previous testing losses')

print('OLD (PREVIOUS) LOSSES:')
plt_losses(prev_tlosses, prev_vlosses, split_axis=False)

In [ ]:
net2 = create_net_2(hidden_size=32, num_layers=3,
                    _ngbr_inner_dense_sizes=[32], _ngbr_out_dense_units=9)
print('net2:');  print(net2)
# reload from saved params
net2.load_parameters('params2/test_net_2_latest.params', ctx=ctx)
print('net2:');  print(net2)

## trainer for net
trainer = create_trainer(net2)
loss_fn = create_loss_fn()

In [ ]:
tcc = train_citycode_list[0]
vcc = test_citycode_list[0]
print('train city:', tcc, cities_names_dict[tcc])
print('valid city:', vcc, cities_names_dict[vcc])

In [ ]:
print(len(tlosses))

# RMSE, MAE, R2

In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams["mathtext.fontset"] = "dejavuserif"

In [ ]:
!cp 'drive/MyDrive/Colab Notebooks Project/presequencing_means_stds_dict.json' .
with open('presequencing_means_stds_dict.json', 'r') as f:
  means_stds_dict = json.load(f)
print(means_stds_dict)

In [ ]:
# unstandardize PM2.5
def _unstdize(ys):
  _pm25mean, _pm25std = means_stds_dict['PM25']
  return (ys*_pm25std)+_pm25mean

## function for R2 from reals and preds
SSE = n * MSE,
or MSE = 1/n * SSE

In [ ]:
def calc_r2(real_ys, pred_ys):
  print(f'  real_ys.shape: {real_ys.shape} ; pred_ys.shape: {pred_ys.shape}')
  _mean_real_y = real_ys.mean()
  _TSS = np.square((real_ys - _mean_real_y)).sum();  # total sum of squares
  _SSE = np.square((real_ys - pred_ys)).sum();  # sum of squared errors
  _R2 = 1 - _SSE / _TSS
  print(f'  mean real y = {_mean_real_y}')
  print(f'  TSS = {_TSS}')
  print(f'  SSE = {_SSE}')
  print(f'    MSE = {_SSE / len(pred_ys)}')
  print(f'    RMSE = {np.sqrt(_SSE / len(pred_ys))}')
  print(f' R2 = SSE / TSS = {_R2}')
  return _R2

## net1

In [ ]:
data = SimpleCityDataSet(together_train_allx_seqs, together_train_y_seqs,
                            together_test_allx_seqs, together_test_y_seqs)

In [ ]:
net = create_net_1(hidden_size=32, num_layers=3)
print('net:');  print(net)
# reload from saved params
net.load_parameters('params2/test_net_1_latest.params', ctx=ctx)
print('net:');  print(net)

## trainer for net
trainer = create_trainer(net)
loss_fn = create_loss_fn()

In [ ]:
train_dataloader: gluon.data.DataLoader = data.get_dataloader(train=True)
val_dataloader: gluon.data.DataLoader = data.get_dataloader(train=False)

In [ ]:
together_all_allx_seqs = np.concatenate([together_train_allx_seqs,together_test_allx_seqs],
                                   axis=0)
print(together_all_allx_seqs.shape)
together_all_y_seqs = np.concatenate([together_train_y_seqs,together_test_y_seqs],
                                   axis=0)
print(together_all_y_seqs.shape)

In [ ]:
# net1's prediction!
# note, batch is entire dataset's together x sequences
_batch = mx.nd.array(together_all_allx_seqs, ctx=ctx, dtype='float32')
_label = mx.nd.array(together_all_y_seqs, ctx=ctx, dtype='float32')
# _batch = _batch.astype('float32')
# _label = _label.astype('float32')
_batch = _batch.as_in_context(ctx)
_label = _label.as_in_context(ctx)
npx.set_np()
pred1 = net(_batch.as_np_ndarray())  # as_np_ndarray() to satiate an error
print('loss:', loss_fn(pred1, _label.as_np_ndarray()).mean())
print('loss:', loss_fn(pred1.flatten(), _label.as_np_ndarray().flatten()).mean())
print('L2Loss (MSE):', gluon.loss.L2Loss()(pred1, _label.as_np_ndarray()).mean())
print('L1Loss (MAE):', gluon.loss.L1Loss()(pred1, _label.as_np_ndarray()).mean())
print(pred1.shape)
print(pred1[0:3])
print(_label[0:3])
print()
pred1 = pred1.asnumpy()
real1 = _label.asnumpy()
pred1 = pred1.flatten()
real1 = real1.flatten()
print(pred1[0:3])
print(real1[0:3])
print()

r2_1 = calc_r2(real1,pred1)
print(r2_1)

In [ ]:
plt.figure(figsize=(5,5))
ax = plt.gca()

_x = _unstdize(pred1)
_y = _unstdize(real1)

ax.scatter(x=_x, y=_y, s=4, alpha=0.4)
ax.axline((0,0), slope=1, color='gray', linestyle='solid', linewidth=1)

# limmax = max(_x.max(), _y.max())
# ax.set_ylim(0,limmax)
# ax.set_xlim(0,limmax)
ax.axis('equal')
ax.axis('square')
ax.set_ylabel('Actual $\\rm PM_{2.5}$')
ax.set_xlabel('Predicted $\\rm PM_{2.5}$')
ax.set_title('(a) NEP-LSTM v. 1 k=3 Actual vs Predicted')

# line of best fit
_lob = np.poly1d(np.polyfit(_x, _y, deg=1))
print(_lob)
ax.axline((0,_lob[0]), slope=_lob[1], color='orange', linestyle='dashed', linewidth=1.5)

## net2

In [ ]:
data2 = ComplexCityDataSet(separate_train_targetx_seqs, separate_train_neighborx_seqs, separate_train_y_seqs,
                            separate_test_targetx_seqs, separate_test_neighborx_seqs, separate_test_y_seqs)

In [ ]:
net2 = create_net_2(hidden_size=32, num_layers=3,
                    _ngbr_inner_dense_sizes=[32], _ngbr_out_dense_units=9)
print('net2:');  print(net2)
# reload from saved params
net2.load_parameters('params2/test_net_2_latest.params', ctx=ctx)
print('net2:');  print(net2)

## trainer for net
trainer = create_trainer(net2)
loss_fn = create_loss_fn()

In [ ]:
train_dataloader2: gluon.data.DataLoader = data2.get_dataloader(train=True)
val_dataloader2: gluon.data.DataLoader = data2.get_dataloader(train=False)

In [ ]:
separate_all_targetx_seqs = np.concatenate([separate_train_targetx_seqs,separate_test_targetx_seqs],
                                   axis=0)
print(separate_all_targetx_seqs.shape)
separate_all_neighborx_seqs = np.concatenate([separate_train_neighborx_seqs,separate_test_neighborx_seqs],
                                   axis=0)
print(separate_all_neighborx_seqs.shape)
separate_all_y_seqs = np.concatenate([separate_train_y_seqs,separate_test_y_seqs],
                                   axis=0)
print(separate_all_y_seqs.shape)

In [ ]:
# net2's prediction!
# note, batch is entire dataset's together x sequences
_batchtgt = mx.nd.array(separate_all_targetx_seqs, ctx=ctx, dtype='float32')
_batchngbr = mx.nd.array(separate_all_neighborx_seqs, ctx=ctx, dtype='float32')
_label = mx.nd.array(separate_all_y_seqs, ctx=ctx, dtype='float32')
# _batch = _batch.astype('float32')
# _label = _label.astype('float32')
_batchtgt = _batchtgt.as_in_context(ctx)
_batchngbr = _batchngbr.as_in_context(ctx)
_label = _label.as_in_context(ctx)
npx.set_np()
pred2 = net2(_batchtgt.as_np_ndarray(), _batchngbr.as_np_ndarray())  # as_np_ndarray() to satiate an error
print('loss:', loss_fn(pred2, _label.as_np_ndarray()).mean())
print('loss:', loss_fn(pred2.flatten(), _label.as_np_ndarray().flatten()).mean())
print('L2Loss (MSE):', gluon.loss.L2Loss()(pred2, _label.as_np_ndarray()).mean())
print('L1Loss (MAE):', gluon.loss.L1Loss()(pred2, _label.as_np_ndarray()).mean())
print(pred2.shape)
print(pred2[0:3])
print(_label[0:3])
print()
pred2 = pred2.asnumpy()
real2 = _label.asnumpy()
pred2 = pred2.flatten()
real2 = real2.flatten()
print(pred2[0:3])
print(real2[0:3])
print()

r2_2 = calc_r2(real2,pred2)
print(r2_2)

In [ ]:
plt.figure(figsize=(5,5))
ax = plt.gca()

_x = _unstdize(pred2)
_y = _unstdize(real2)

ax.scatter(x=_x, y=_y, s=4, alpha=0.4, color='tab:purple')
ax.axline((0,0), slope=1, color='gray', linestyle='solid', linewidth=1)

# limmax = max(_x.max(), _y.max())
# ax.set_ylim(0,limmax)
# ax.set_xlim(0,limmax)
ax.axis('equal')
ax.axis('square')
ax.set_ylabel('Actual $\\rm PM_{2.5}$')
ax.set_xlabel('Predicted $\\rm PM_{2.5}$')
ax.set_title('(a) Model 2 n=3 Actual vs Predicted')

# line of best fit
_lob = np.poly1d(np.polyfit(_x, _y, deg=1))
print(_lob)
ax.axline((0,_lob[0]), slope=_lob[1], color='orange', linestyle='dashed', linewidth=1.5)

In [ ]:
print(r2_1)
print(r2_2)

### Plot both

In [ ]:
plt.figure(figsize=(5,5))
fig, (ax1, ax2) = plt.subplots(1, 2)

_x1 = _unstdize(pred1)
_y1 = _unstdize(real1)

ax1.scatter(x=_x1, y=_y1, s=4, alpha=0.4, color='tab:blue')
ax1.axline((0,0), slope=1, color='gray', linestyle='solid', linewidth=1)

# limmax = max(_x1.max(), _y1.max())
# ax1.set_ylim(0,limmax)
# ax1.set_xlim(0,limmax)
ax1.axis('equal')
ax1.axis('square')
ax1.set_ylabel('Actual $\\rm PM_{2.5}$ $(\\mu g/m^3)$')
ax1.set_xlabel('Predicted $\\rm PM_{2.5}$ $(\\mu g/m^3)$')
ax1.set_title('(a) NEP-LSTM variant 1')

# line of best fit
_lob1 = np.poly1d(np.polyfit(_x1, _y1, deg=1))
print(_lob1)
ax1.axline((0,_lob1[0]), slope=_lob1[1], color='orange', linestyle='dashed', linewidth=1.5)



_x2 = _unstdize(pred2)
_y2 = _unstdize(real2)

ax2.scatter(x=_x2, y=_y2, s=4, alpha=0.4, color='tab:purple')
ax2.axline((0,0), slope=1, color='gray', linestyle='solid', linewidth=1)

# limmax = max(_x2.max(), _y2.max())
# ax2.set_ylim(0,limmax)
# ax2.set_xlim(0,limmax)
ax2.axis('equal')
ax2.axis('square')
ax2.set_ylabel('Actual $\\rm PM_{2.5}$ $(\\mu g/m^3)$')
ax2.set_xlabel('Predicted $\\rm PM_{2.5}$ $(\\mu g/m^3)$')
ax2.set_title('(b) NEP-LSTM variant 2')

# line of best fit
_lob2 = np.poly1d(np.polyfit(_x2, _y2, deg=1))
print(_lob2)
ax2.axline((0,_lob2[0]), slope=_lob2[1], color='orange', linestyle='dashed', linewidth=1.5)



ax1.text(0.05, 0.95,
         '$\\rm MSE = 0.200$' +'\n'+ '$\\rm R^{2} = '+f'{r2_1:.3f}$',
         horizontalalignment='left', verticalalignment='top', transform=ax1.transAxes)
ax2.text(0.05, 0.95,
         '$\\rm MSE = 0.231$' +'\n'+ '$\\rm R^{2} = '+f'{r2_2:.3f}$',
         horizontalalignment='left', verticalalignment='top', transform=ax2.transAxes)


# Feature importance by permutation

In [ ]:
together_all_allx_seqs = np.concatenate([together_train_allx_seqs,together_test_allx_seqs],
                                   axis=0)
print(together_all_allx_seqs.shape)
together_all_y_seqs = np.concatenate([together_train_y_seqs,together_test_y_seqs],
                                   axis=0)
print(together_all_y_seqs.shape)

In [ ]:
separate_all_targetx_seqs = np.concatenate([separate_train_targetx_seqs,separate_test_targetx_seqs],
                                   axis=0)
print(separate_all_targetx_seqs.shape)
separate_all_neighborx_seqs = np.concatenate([separate_train_neighborx_seqs,separate_test_neighborx_seqs],
                                   axis=0)
print(separate_all_neighborx_seqs.shape)
separate_all_y_seqs = np.concatenate([separate_train_y_seqs,separate_test_y_seqs],
                                   axis=0)
print(separate_all_y_seqs.shape)

In [ ]:
# ensure that copying the array will not modify the original
print(separate_all_targetx_seqs[1])
_xcopy = separate_all_targetx_seqs.copy()
_xcopy[:,:,2] = 0
print(print(_xcopy[1]))

In [ ]:
def feature_importance(net, inputs, labely, count=10):
  assert isinstance(inputs, (tuple, list)), 'provide a list of one or more inputs to the net'
  for inp in inputs:
    print(inp.shape)
  total_features = sum([inp.shape[2] for inp in inputs])
  print('total features =', total_features)
  print('labely.shape:', labely.shape)
  rng = np.random.default_rng()

  MSEs = {fi: [] for fi in range(total_features)}
  MAEs = {fi: [] for fi in range(total_features)}
  R2s =  {fi: [] for fi in range(total_features)}

  # provide the defaults

  pred = net(*[mx.nd.array(inp,
                  ctx=ctx, dtype='float32') \
                  .as_np_ndarray()
              for inp in inputs])
  label_to_use = mx.nd.array(labely,
                    ctx=ctx, dtype='float32') \
                    .as_np_ndarray()
  mse = gluon.loss.L2Loss()(pred, label_to_use).mean()
  mae = gluon.loss.L1Loss()(pred, label_to_use).mean()
  print('L2Loss (MSE):', mse)
  print('L1Loss (MAE):', mae)
  r2 = calc_r2(label_to_use.asnumpy().flatten(),
                pred.asnumpy().flatten())
  orig_MSE = float(mse)
  orig_MAE = float(mae)
  orig_R2 = float(r2)

  # go and permute

  for ci in range(count):
    print()
    print('COUNT:', ci)

    for fi in range(total_features):
        print('fi:', fi)

        # permute an input and prepare the inputs
        inputs_to_use = []
        _featct = 0

        for inp in inputs:
          inpprepped = None
          # permute feature at index fi in the input
          _feats = inp.shape[2]
          print('_feats =', _feats)
          _potential_fi = fi - _featct
          if 0 <= _potential_fi < _feats:
            print('permuting, _potential_fi =',_potential_fi)
            inpprepped = inp.copy()
            # permute the slice at feature index _potential_fi
            inpprepped[:,:,_potential_fi] = rng.permuted(
                inpprepped[:,:,_potential_fi],
                axis=0
            )
          else:
            inpprepped = inp
          # prepare input
          inpprepped = mx.nd.array(inpprepped,
                                   ctx=ctx, dtype='float32') \
                                   .as_np_ndarray()  # satiates an error
          # go to next input
          inputs_to_use.append(inpprepped)
          _featct += _feats
        
        # now, inputs_to_use is populated
        assert len(inputs_to_use) == len(inputs)

        label_to_use = mx.nd.array(labely,
                                   ctx=ctx, dtype='float32') \
                                   .as_np_ndarray()  # satiates an error
        # predict
        pred = net(*inputs_to_use)
        mse = gluon.loss.L2Loss()(pred, label_to_use).mean()
        mae = gluon.loss.L1Loss()(pred, label_to_use).mean()
        print('L2Loss (MSE):', mse)
        print('L1Loss (MAE):', mae)
        r2 = calc_r2(label_to_use.asnumpy().flatten(),
                     pred.asnumpy().flatten())
        MSEs[fi].append(float(mse))
        MAEs[fi].append(float(mae))
        R2s[fi].append(float(r2))

    # end of count

  # done with all counts
  return orig_MSE, orig_MAE, orig_R2, MSEs, MAEs, R2s

In [ ]:
orig_MSE, orig_MAE, orig_R2, MSEs, MAEs, R2s = \
feature_importance(net, inputs=(together_all_allx_seqs,),
                   labely=together_all_y_seqs, count=10)

In [ ]:
print(orig_MSE, '--', MSEs)
print(orig_MAE, '--', MAEs)
print(orig_R2, '--', R2s)

In [ ]:
featnames = ['$PM_{10}$', 'CO', '$SO_2$', '$O_3$', '$NO_2$', 'WS'] \
            + ['$PM_{2.5}$ n #0', '$PM_{10}$ n #0', 'CO n #0', '$SO_2$ n #0', '$O_3$ n #0',
               '$NO_2$ n #0', 'WS n #0', 'WD.max n #0', 'distance n #0'] \
            + ['$PM_{2.5}$ n #1', '$PM_{10}$ n #1', 'CO n #1', '$SO_2$ n #1', '$O_3$ n #1',
               '$NO_2$ n #1', 'WS n #1', 'WD.max n #1', 'distance n #1'] \
            + ['$PM_{2.5}$ n #2', '$PM_{10}$ n #2', 'CO n #2', '$SO_2$ n #2', '$O_3$ n #2',
               '$NO_2$ n #2', 'WS n #2', 'WD.max n #2', 'distance n #2']

In [ ]:
for fi in MSEs.keys():
    plt.plot([*zip(np.array(MSEs[fi])-orig_MSE,
                   np.array(MAEs[fi])-orig_MAE,
                   np.array(R2s[fi])-orig_R2
                   )],
             label=f'fi {fi}')
plt.show()

fig = plt.figure(figsize=(8,8))
ax = fig.gca()
ax.set_title('MSEs importances:')
keys = list(MSEs.keys())
ax.barh(range(len(MSEs)),
        [
            (np.array(MSEs[fi])-orig_MSE).mean()
            for fi in keys
        ],
        xerr=[
            (np.array(MSEs[fi])-orig_MSE).std()
            for fi in keys
        ]
         )
ax.set_yticks(range(len(MSEs)))
_=ax.set_yticklabels([featnames[key] for key in keys])

In [ ]:
orig_MSE, orig_MAE, orig_R2, MSEs, MAEs, R2s = \
feature_importance(net2, inputs=(separate_all_targetx_seqs, separate_all_neighborx_seqs),
                   labely=together_all_y_seqs, count=10)

In [ ]:
print(orig_MSE, '--', MSEs)
print(orig_MAE, '--', MAEs)
print(orig_R2, '--', R2s)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.gca()
ax.set_title('MSEs importances:')
keys = list(MSEs.keys())
ax.barh(range(len(MSEs)),
        [
            (np.array(MSEs[fi])-orig_MSE).mean()
            for fi in keys
        ],
        xerr=[
            (np.array(MSEs[fi])-orig_MSE).std()
            for fi in keys
        ]
         )
ax.set_yticks(range(len(MSEs)))
_=ax.set_yticklabels([featnames[key] for key in keys])